In [1]:
# Used for historical weather data
# https://frost.met.no/index.html

import requests
import pandas as pd

# client secret
# f7a1025d-1588-4315-b008-3aee471491fa

client_id = 'bc35bf94-91a2-4582-af49-78957d8c4e0a' # client ID I got from frost.met.api.no


# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'

parameters = {
    'sources': 'SN18700,SN90450', # Henter data fra Oslo blindern
    'elements': 'sum(precipitation_amount P1D),mean(wind_speed P1D)', # PID = 1 dag (ISO 8601)
    'referencetime': '2010-04-01/2010-04-03',
}

In [2]:
# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=(client_id,''))

# Extract JSON data
json_data = r.json()


# Check if the request worked, print out any errors
if r.status_code == 200:
    json_data = json_data['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json_data['error']['message'])
    print('Reason: %s' % json_data['error']['reason'])

print(json_data)

Data retrieved from frost.met.no!
[{'sourceId': 'SN18700:0', 'referenceTime': '2010-04-01T00:00:00.000Z', 'observations': [{'elementId': 'sum(precipitation_amount P1D)', 'value': 13.5, 'unit': 'mm', 'timeOffset': 'PT18H', 'timeResolution': 'P1D', 'timeSeriesId': 0, 'performanceCategory': 'C', 'exposureCategory': '2', 'qualityCode': 2}, {'elementId': 'sum(precipitation_amount P1D)', 'value': 29, 'unit': 'mm', 'timeOffset': 'PT6H', 'timeResolution': 'P1D', 'timeSeriesId': 0, 'performanceCategory': 'C', 'exposureCategory': '2', 'qualityCode': 2}, {'elementId': 'mean(wind_speed P1D)', 'value': 1.7, 'unit': 'm/s', 'level': {'levelType': 'height_above_ground', 'unit': 'm', 'value': 10}, 'timeOffset': 'PT0H', 'timeResolution': 'P1D', 'timeSeriesId': 0, 'performanceCategory': 'C', 'exposureCategory': '2', 'qualityCode': 2}]}, {'sourceId': 'SN18700:0', 'referenceTime': '2010-04-02T00:00:00.000Z', 'observations': [{'elementId': 'sum(precipitation_amount P1D)', 'value': 3.3, 'unit': 'mm', 'timeOf

In [3]:
# This will return a Dataframe with all of the observations in a table format
df = pd.DataFrame()
for i in range(len(json_data)):
    row = pd.DataFrame(json_data[i]['observations'])
    row['referenceTime'] = json_data[i]['referenceTime']
    row['sourceId'] = json_data[i]['sourceId']
    df = pd.concat([df, row], ignore_index=True)
df = df.reset_index()

df.head()

,index,elementId,value,unit,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,level,referenceTime,sourceId
0,0,sum(precipitation_amount P1D),13.5,mm,PT18H,P1D,0,C,2,2,NaN,2010-04-01T00:00:00.000Z,SN18700:0
1,1,sum(precipitation_amount P1D),29.0,mm,PT6H,P1D,0,C,2,2,NaN,2010-04-01T00:00:00.000Z,SN18700:0
2,2,mean(wind_speed P1D),1.7,m/s,PT0H,P1D,0,C,2,2,"{'levelType': 'height_above_ground', 'unit': '...",2010-04-01T00:00:00.000Z,SN18700:0
3,3,sum(precipitation_amount P1D),3.3,mm,PT18H,P1D,0,C,2,2,NaN,2010-04-02T00:00:00.000Z,SN18700:0
4,4,sum(precipitation_amount P1D),9.0,mm,PT6H,P1D,0,C,2,2,NaN,2010-04-02T00:00:00.000Z,SN18700:0
